In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
from scipy import stats
import datetime 
import psycopg2
from psycopg2 import sql
import sqlalchemy as db
import os
import chardet

In [2]:
# Basic function to create a cursor to the database
def create_cursor():
    db_name = "Addidas_Webstore_shoe_data"
    db_user = "postgres"
    db_password = "userDGL"
    db_host = "localhost"  # Change if using a remote host
    db_port = "5432"       # Default PostgreSQL port

    try:
        conn = psycopg2.connect(host=db_host,database=db_name, user=db_user, password=db_password, port=db_port)
        conn.rollback() # Reset the connection to solve bug with the cursor
    except:
        print("I am unable to connect to the database")
    cur = conn.cursor()

    return conn, cur

In [3]:
# Basic function to close a cursor to the database
def close_cursor(conn, cur):
    cur.close()
    conn.close()

In [4]:
# Database connection details
db_name = "Addidas_Webstore_shoe_data"
db_user = "postgres"
db_password = "userDGL"
db_host = "localhost"  # Change if using a remote host
db_port = "5432"       # Default PostgreSQL port

# Create a connection string for SQLAlchemy
connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = db.create_engine(connection_string)

In [5]:
# Look at REAME.me file for information regarding the csv files
# Get the list of the csv files in the database
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, "../.."))
csv_file_path = parent_directory+"\\data\\external"
list_csv_files = os.listdir(csv_file_path)

In [6]:
# Generate the tables in the psql database from the csv files 
for csv_file_name in list_csv_files:
    table_name = csv_file_name.split(".")[0]
    full_path = os.path.join(csv_file_path, csv_file_name)

    # Read cleaned file
    df = pd.read_csv(full_path, encoding="utf-8") 
    df.to_sql(table_name, engine, index=False, if_exists='replace')

In [7]:
inspector = db.inspect(engine)
table_names = inspector.get_table_names()

In [8]:
table_names

['shoes_dim', 'shoes_fact', 'country_dim', 'sales_info']

In [9]:
conn, cur = create_cursor() 

query = f"""--sql
SELECT * FROM shoes_fact;
"""
cur.execute(query)
results = cur.fetchall()
column_names = [desc[0] for desc in cur.description]    
pd.DataFrame(results, columns=column_names)

,Unnamed: 0,id,price,category,size,availability,date,country_code
0,63575,HP9426,60.0,sneakers,36,0,07/01/2025,DE
1,63576,HP9426,60.0,sneakers,36 2/3,0,07/01/2025,DE
2,63577,HP9426,60.0,sneakers,37 1/3,0,07/01/2025,DE
3,63578,HP9426,60.0,sneakers,38,0,07/01/2025,DE
4,63579,HP9426,60.0,sneakers,38 2/3,1,07/01/2025,DE
...,...,...,...,...,...,...,...,...
299151,847278,JI4476,160.0,sneakers,47 1/3,3,2025-01-16,BE
299152,847279,JI4476,160.0,sneakers,48,0,2025-01-16,BE
299153,847280,JI4476,160.0,sneakers,48 2/3,0,2025-01-16,BE
299154,847281,JI4476,160.0,sneakers,49 1/3,2,2025-01-16,BE


In [10]:
# Change the name of a column that was originally set as unknown
conn, cur = create_cursor()
try:
    query = sql.SQL("ALTER TABLE {} RENAME COLUMN {} TO {};").format(
        sql.Identifier('shoes_fact'),   # Table name
        sql.Identifier('Unnamed: 0'),     # Old column name
        sql.Identifier('number')         # New column name
    )

    # Execute the query
    cur.execute(query)

    # Commit the changes (required to save alterations in the database)
    conn.commit()

    close_cursor(conn, cur)
except:
    print("Table already modified")

In [11]:
# Shoes_fact table
conn, cur = create_cursor()

cur.execute("SELECT * FROM {}".format(table_names[3]))
results = cur.fetchall()
columns = [desc[0] for desc in cur.description]
df_first_table = pd.DataFrame(results, columns=columns)
df_first_table.head()

,id,price,category,size,country_code,best_for_wear,gender,dominant_color,sub_color1,sub_color2,currency,shoe_metric
0,HP9426,60.0,sneakers,36,DE,City,U,Cloud White,Core Black,Cloud White,euro,eu
1,HP9426,60.0,sneakers,36,DE,City,U,Cloud White,Core Black,Cloud White,euro,eu
2,HP9426,60.0,sneakers,36 2/3,DE,City,U,Cloud White,Core Black,Cloud White,euro,eu
3,HP9426,60.0,sneakers,36 2/3,DE,City,U,Cloud White,Core Black,Cloud White,euro,eu
4,HP9426,60.0,sneakers,37 1/3,DE,City,U,Cloud White,Core Black,Cloud White,euro,eu


In [12]:
# Shoes_dim table
conn, cur = create_cursor()

cur.execute("SELECT * FROM {}".format(table_names[1]))
results = cur.fetchall()
columns = [desc[0] for desc in cur.description]
df_first_table = pd.DataFrame(results, columns=columns)
df_first_table.head()

,number,id,price,category,size,availability,date,country_code
0,63575,HP9426,60.0,sneakers,36,0,07/01/2025,DE
1,63576,HP9426,60.0,sneakers,36 2/3,0,07/01/2025,DE
2,63577,HP9426,60.0,sneakers,37 1/3,0,07/01/2025,DE
3,63578,HP9426,60.0,sneakers,38,0,07/01/2025,DE
4,63579,HP9426,60.0,sneakers,38 2/3,1,07/01/2025,DE


In [13]:
# Country_dim 
conn, cur = create_cursor()

cur.execute("SELECT * FROM {}".format(table_names[2]))
results = cur.fetchall()
columns = [desc[0] for desc in cur.description]
df_first_table = pd.DataFrame(results, columns=columns)
df_first_table.head()

,country_code,currency,shoe_metric
0,DE,euro,eu
1,US,usd,usa
2,BE,euro,eu
3,UK,pounds,uk


In [14]:
# Check for duplicates in the tables 
for table_name in ['country_dim', 'shoes_dim', 'shoes_fact']:
    print("\n",table_name)
    conn, cur = create_cursor()

    cur.execute("""
                SELECT column_name
                FROM information_schema.columns
                WHERE table_name = %s AND table_schema = 'public';
            """, (table_name,))
            
    columns = [row[0] for row in cur.fetchall()]

    group_by_clause = ", ".join(columns)  # Dynamically create GROUP BY clause
    query = f"""
    SELECT {group_by_clause}, COUNT(*)
    FROM {table_name}
    GROUP BY {group_by_clause}
    HAVING COUNT(*) > 1;
    """
    # Execute the duplicate query
    cur.execute(query)
    duplicates = cur.fetchall()

    # Print results
    if duplicates:
        print("Duplicate rows found:")
        for row in duplicates:
            print(row)
    else:
        print("No duplicate rows found.")
    
    close_cursor(conn, cur)


 country_dim
No duplicate rows found.

 shoes_dim
No duplicate rows found.

 shoes_fact
No duplicate rows found.


In [15]:
# Checking for null values in the tables
for table_name in ['country_dim', 'shoes_dim', 'shoes_fact']:
    conn, cur = create_cursor()
    print(table_name)
    cur.execute("""--sql
        SELECT column_name
        FROM information_schema.columns
        WHERE table_name = %s AND table_schema = 'public';
    """, (table_name,))

    columns = [row[0] for row in cur.fetchall()]

    # Check for NULL values in each column
    null_counts = {}
    for column in columns:
        cur.execute(f"""--sql
            SELECT COUNT(*) 
            FROM {table_name} 
            WHERE {column} IS NULL;
        """)
        count = cur.fetchone()[0]
        null_counts[column] = count

    # Print results
    print(f"NULL value counts in table '{table_name}':")
    for column, count in null_counts.items():
        print(f"{column}: {count}")
    print("\n")

country_dim
NULL value counts in table 'country_dim':
country_code: 0
currency: 0
shoe_metric: 0


shoes_dim
NULL value counts in table 'shoes_dim':
id: 0
name: 0
best_for_wear: 0
gender: 0
image_url: 0
dominant_color: 0
sub_color1: 2
sub_color2: 4


shoes_fact
NULL value counts in table 'shoes_fact':
number: 0
id: 0
price: 0
category: 0
size: 0
availability: 0
date: 0
country_code: 0




In [16]:
# Check if shoes_fact
conn, cur = create_cursor()

query = f"""--sql
SELECT id, price FROM shoes_fact where price < 0
;"""

cur.execute(query)
output = cur.fetchall()
pd.DataFrame(output, columns=['id', 'price'])

,id,price


In [17]:
# Check the value counts of availability in shoes_fact.availability --> recommend by the kaggle dataset
conn, cur = create_cursor()

query = f"""--sql
SELECT availability, count(*) as value_count
FROM shoes_fact
group by availability
order by value_count desc;
;"""

cur.execute(query)
output = cur.fetchall()
pd.DataFrame(output, columns=['availability', 'value_count'])

,availability,value_count
0,0,135262
1,15,91811
2,1,17135
3,2,8827
4,3,6665
5,4,5382
6,5,4741
7,6,4090
8,7,4081
9,8,3803


In [18]:
#  Check the value counts of category in shoes_fact
conn, cur = create_cursor()

query = f"""--sql
SELECT category, count(*) as value_count
FROM shoes_fact
group by category
order by value_count desc;
"""

cur.execute(query)
output = cur.fetchall()
pd.DataFrame(output, columns=['category', 'value_count'])

,category,value_count
0,running-shoes,78955
1,sneakers,74676
2,outdoor-shoes,44314
3,walking-shoes,31583
4,us/athletic_sneakers,20183
5,athletic_sneakers,10191
6,tennis-shoes,9518
7,gym_training-shoes,7911
8,football-shoes,4518
9,us/walking-shoes,4157


In [19]:
# Check the categories for the US store only
conn, cur = create_cursor()

query = f"""--sql
SELECT category, count(*) as value_count
FROM shoes_fact
WHERE country_code = 'US'
GROUP BY category
ORDER BY value_count DESC;
"""

cur.execute(query)
output = cur.fetchall()
pd.DataFrame(output, columns=['category', 'value_count'])

,category,value_count
0,us/athletic_sneakers,20183
1,athletic_sneakers,10191
2,us/walking-shoes,4157
3,running-shoes,3970
4,us/running-shoes,3896
5,us/soccer-shoes,3789
6,occer-shoes,2063
7,walking-shoes,1717
8,us/workout-shoes,1402
9,us/hiking-shoes,863


In [20]:
# Check the categories for the other stores
conn, cur = create_cursor()

query = f"""--sql
SELECT category, count(*) as value_count
FROM shoes_fact
WHERE country_code != 'US'
GROUP BY category
ORDER BY value_count DESC;
"""

cur.execute(query)
output = cur.fetchall()
pd.DataFrame(output, columns=['category', 'value_count'])

,category,value_count
0,running-shoes,74985
1,sneakers,74676
2,outdoor-shoes,44314
3,walking-shoes,29866
4,tennis-shoes,9518
5,gym_training-shoes,7911
6,football-shoes,4518


There are shoes in the US store that not necessarily have the US\ in front of it's category, meaning that this information is redundant.

In [21]:
# Change the categories that start with'us/'
conn, cur = create_cursor()

query = f"""--sql
UPDATE shoes_fact
SET category = substring(category FROM 4)
where category like 'us/%';
"""

cur.execute(query)

query = f"""--sql
SELECT category, count(*) as value_count
FROM shoes_fact
WHERE country_code = 'US'
GROUP BY category
ORDER BY value_count DESC;
"""

cur.execute(query)
output = cur.fetchall()
pd.DataFrame(output, columns=['category', 'value_count'])

,category,value_count
0,athletic_sneakers,30374
1,running-shoes,7866
2,walking-shoes,5874
3,soccer-shoes,3789
4,occer-shoes,2063
5,hiking-shoes,1726
6,workout-shoes,1402
7,tennis-shoes,274


In [22]:
# Check the categories for the other stores
conn, cur = create_cursor()

query = f"""--sql
SELECT size, count(*) as value_count
FROM shoes_fact
WHERE country_code != 'US'
GROUP BY size
ORDER BY value_count DESC;
"""

cur.execute(query)
output = cur.fetchall()
pd.DataFrame(output, columns=['size', 'value_count'])

,size,value_count
0,40,13165
1,39 1/3,13123
2,38 2/3,11176
3,42,10921
4,42 2/3,10875
...,...,...
115,15,3
116,16,3
117,55 2/3,3
118,17,3


In [23]:
# Check genders in shoes_dim
conn, cur = create_cursor()

query = f"""--sql
SELECT gender, count(*) as value_count
from shoes_dim
GROUP BY gender
ORDER BY value_count DESC;
"""

cur.execute(query)
results = cur.fetchall()
columns = [desc[0] for desc in cur.description]
df_first_table = pd.DataFrame(results, columns=columns)
df_first_table

,gender,value_count
0,U,1134
1,W,1014
2,K,713
3,M,441


In [24]:
# Check genders in shoes_dim
conn, cur = create_cursor()

query = f"""--sql
SELECT best_for_wear,  count(*) as value_count
from shoes_dim
GROUP BY best_for_wear
ORDER BY value_count DESC;
"""

cur.execute(query)
results = cur.fetchall()
print(np.shape(results))
columns = [desc[0] for desc in cur.description]
pd.DataFrame(results, columns=columns)

(45, 2)


,best_for_wear,value_count
0,Neutral,1153
1,Racing,304
2,Everyday,222
3,Comfort,182
4,Walking,180
5,Day Hiking,123
6,City,112
7,Off-Court,108
8,Padel Tennis,105
9,Stability,53


In [25]:
# Check genders in shoes_dim
conn, cur = create_cursor()

query = f"""--sql
SELECT name,  count(*) as value_count
from shoes_dim
GROUP BY name
ORDER BY value_count DESC;
"""

cur.execute(query)
results = cur.fetchall()
print(np.shape(results))
columns = [desc[0] for desc in cur.description]
pd.DataFrame(results, columns=columns)

(1055, 2)


,name,value_count
0,Dropset 3 strength training shoes,33
1,Runfalcon 5 Running Shoes,30
2,Supernova Rise Running Shoes,29
3,Ultraboost 5X Shoes,28
4,Ultraboost 1.0 Shoes,25
...,...,...
1050,Anthony Edwards 1 Low Iron Metallic Basketball...,1
1051,Adizero Avanti Tyo Track and Field Lightstrike...,1
1052,Forum Mid Shoes,1
1053,Terrex Disney Snowpitch Cold.Rdy Winter Wander...,1


In [26]:
# Check genders in shoes_dim
conn, cur = create_cursor()

query = f"""--sql
SELECT dominant_color,  count(*) as value_count
from shoes_dim
GROUP BY dominant_color
ORDER BY value_count DESC;
"""

cur.execute(query)
results = cur.fetchall()
print(np.shape(results))
columns = [desc[0] for desc in cur.description]
pd.DataFrame(results, columns=columns)

(204, 2)


,dominant_color,value_count
0,Core Black,985
1,Cloud White,720
2,Olive Strata,66
3,Off White,56
4,Lucid Red,46
...,...,...
199,Warm Vanilla,1
200,Supplier Colour,1
201,Lucid Cyan,1
202,Preloved Ruby,1


In [27]:
# Check for outliers in price 
conn, cur = create_cursor()

query = f"""--sql
SELECT price, count(*) as value_count
FROM shoes_fact
GROUP BY price
order by price desc;
"""

cur.execute(query)
results = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(results, columns=columns)

,price,value_count
0,500.00,220
1,350.00,380
2,314.00,17
3,300.00,2493
4,280.00,232
...,...,...
249,20.79,24
250,20.00,760
251,19.00,80
252,18.00,168


In [28]:
table_names

['shoes_dim', 'shoes_fact', 'country_dim', 'sales_info']

In [29]:
# Fetch column names from table_names[1]
conn, cur = create_cursor()

try:
    query = """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = %s AND table_schema = %s;
    """
    cur.execute(query, ('shoes_fact', 'public'))
    columns_table_2 = [row[0] for row in cur.fetchall()]
except Exception as e:
    print("Error occurred:", e)

# Fetch column names from table_names[0]
cur.execute("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = 'shoes_dim' AND table_schema = 'public';
""")
columns_table_1 = [row[0] for row in cur.fetchall()]

cur.execute("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = %s AND table_schema = %s;
""", (table_names[0], 'public'))
columns_table_0 = [row[0] for row in cur.fetchall()]

print("Columns in table_names[2]:", columns_table_2)
print("Columns in table_names[1]:", columns_table_1)
print("Columns in table_names[0]:", columns_table_0)

Columns in table_names[2]: ['number', 'id', 'price', 'category', 'size', 'availability', 'date', 'country_code']
Columns in table_names[1]: ['id', 'name', 'best_for_wear', 'gender', 'image_url', 'dominant_color', 'sub_color1', 'sub_color2']
Columns in table_names[0]: ['id', 'name', 'best_for_wear', 'gender', 'image_url', 'dominant_color', 'sub_color1', 'sub_color2']


In [30]:
conn, cur = create_cursor()
with conn:
    with conn.cursor() as cur:
        query = """--sql
        SELECT name, price
        FROM shoes_fact
        LEFT JOIN shoes_dim ON shoes_fact.id = shoes_dim.id
        WHERE price > %s
        group by name, price
        order by price desc;
        """
        cur.execute(query, (280,))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(results, columns=columns)
        print(df)

                                                 name  price
0                             Adizero Adios Pro Evo 1  500.0
1                                 Dingyun Zhang Kouza  350.0
2                                   Y-3 Takumi Sen 10  350.0
3                                         Y-3 Country  350.0
4                                         Y-3 Country  314.0
5                Adizero Prime X 2.0 STRUNG Laufschuh  300.0
6                        4DFWD x STRUNG Running Shoes  300.0
7   adidas by Stella McCartney x Terrex Free Hiker...  300.0
8                            4DFWD x STRUNG Laufschuh  300.0
9   adidas by Stella McCartney x Terrex Free Hiker...  300.0
10                            F50+ Firm Ground Cleats  300.0
11           Adizero Prime X 2.0 STRUNG Running Shoes  300.0
12                      ADIZERO PRIME SP3 STRUNG X Y3  300.0
13                   Adizero Prime X 2.0 STRUNG Shoes  300.0


In [31]:
conn, cur = create_cursor()
with conn:
    with conn.cursor() as cur:
        query = f"""--sql
        SELECT country_code, count(*) as value_count
        from shoes_fact
        GROUP BY country_code
        order by value_count desc;
        """
        cur.execute(query, (280,))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(results, columns=columns)
        print(df)

  country_code  value_count
0           DE       216202
1           US        53368
2           BE        26453
3           UK         3133


In [32]:
conn, cur = create_cursor()
with conn:
    with conn.cursor() as cur:
        query = f"""--sql
        SELECT date, count(*) as value_count
        from shoes_fact
        GROUP BY date
        order by value_count desc;
        """
        cur.execute(query, (280,))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(results, columns=columns)
        print(df)

          date  value_count
0   14/01/2025        48665
1   2025-01-16        35485
2   11/01/2025        25971
3   08/01/2025        25960
4   07/01/2025        25779
5   10/01/2025        25505
6   2025-01-15        25284
7   12/01/2025        24902
8   15/01/2025        22864
9   09/01/2025        20567
10  13/01/2025        18174


In [33]:
conn, cur = create_cursor()

query = f"""--sql
CREATE TABLE IF NOT EXISTS sales_info AS
SELECT shoes_fact.id, price, category, size, shoes_fact.country_code, best_for_wear, gender, dominant_color, sub_color1, sub_color2, currency, shoe_metric
FROM shoes_fact
JOIN shoes_dim ON shoes_fact.id = shoes_dim.id
JOIN country_dim ON shoes_fact.country_code = country_dim.country_code;
"""

cur.execute(query)

In [34]:
conn, cur = create_cursor() 

with conn:
    with conn.cursor() as cur:
        try:
            # Query to get all table names from the public schema
            query = """
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name;
            """

            cur.execute(query)
            results = cur.fetchall()
            
            # Extract table names into a list
            table_names = [row[0] for row in results]
            print("Tables in the database:", table_names)
            
        except Exception as e:
            print("Error occurred while fetching table names:", e)


Tables in the database: ['country_dim', 'sales_info', 'shoes_dim', 'shoes_fact']
